# Merge Planet and Satellite Ecliptic Coordinates with SSN

This notebook merges the ecliptic longitude and latitude data for planets and satellites with the daily Sunspot Number (SSN) data.
The output is a single Parquet file containing:
- Date
- SSN
- Ecliptic Longitude and Latitude for all processed planets and satellites.

Output file: `../../data/ready/planets_satellites_lonlat.parquet`

In [1]:
import pandas as pd
import numpy as np
import os
import glob

# Custom date parsing function to handle "A.D. YYYY-MMM-DD HH:MM:SS.SSSS" format
def parse_special_date(date_str):
    # Remove "A.D. " prefix
    if isinstance(date_str, str) and "A.D. " in date_str:
        date_str = date_str.replace("A.D. ", "")
    # Parse the remaining part
    try:
        return pd.to_datetime(date_str, format="%Y-%b-%d %H:%M:%S.%f")
    except:
        try:
            return pd.to_datetime(date_str)
        except:
            # print(f"Unable to parse date: {date_str}")
            return pd.NaT
            
# Read Sunspot Number (SSN) data
ssn_path = '../../data/ready/ssn_daily_1849_2025.csv'
df_ssn = pd.read_csv(ssn_path)

# Convert date column to datetime format
df_ssn['date'] = pd.to_datetime(df_ssn['date'])

# Define folders to traverse (Ecliptic Longitude and Latitude)
folder_paths = [
    '../../data/00_raw/helio_ecl_sph_00h/planets_dwarfs_daily_1849',
    '../../data/00_raw/helio_ecl_sph_00h/satellites_daily_1849'
]

# Initialize an empty DataFrame to store merged data
merged_df = None

# Traverse all folders
for folder_path in folder_paths:
    print(f"Processing folder: {folder_path}")
    
    # Get all csv files in the folder
    all_files = glob.glob(os.path.join(folder_path, '*.csv'))
    
    # Traverse all csv files in the folder
    for file in all_files:
        # print(f"Processing file: {file}")
        # Read file
        df_temp = pd.read_csv(file)
        
        # Convert date column to datetime format
        if 'date' in df_temp.columns:
            df_temp['date'] = df_temp['date'].apply(parse_special_date)
        
        # Remove columns with suffix "_r" (distance)
        columns_to_keep = ['date'] + [col for col in df_temp.columns if col != 'date' and not col.endswith('_r')]
        df_temp = df_temp[columns_to_keep]
        
        # If it's the first file, set it as the base DataFrame
        if merged_df is None:
            merged_df = df_temp
        else:
            # Otherwise, merge by date, keeping all columns
            merged_df = pd.merge(merged_df, df_temp, on='date', how='outer')

# Ensure merged_df is not empty
if merged_df is None:
    print("No files found or all files are empty")
else:
    # Ensure date column is the first column
    cols = merged_df.columns.tolist()
    if 'date' in cols:
        cols.remove('date')
        cols = ['date'] + cols
        merged_df = merged_df[cols]

    # Merge SSN data (left merge based on date)
    result_df = pd.merge(merged_df, df_ssn[['date', 'ssn']], on='date', how='left')

    # Adjust column order: date first, ssn second
    cols = result_df.columns.tolist()
    if 'date' in cols:
        cols.remove('date')
    if 'ssn' in cols:
        cols.remove('ssn')
        result_df = result_df[['date', 'ssn'] + cols]
    else:
        print("Warning: ssn column not found after merge")
        result_df = result_df[['date'] + cols]

    # Standardize date format to YYYY-MM-DD
    result_df['date'] = pd.to_datetime(result_df['date']).dt.strftime('%Y-%m-%d')

    # Save result as Parquet in the final directory
    output_path = '../../data/ready/planets_satellites_lonlat.parquet'
    result_df.to_parquet(output_path, index=False)

    print(f"Processing completed. Merged data has {len(result_df)} rows and {len(result_df.columns)} columns.")
    print(f"Saved to: {output_path}")

Processing folder: ../../data/00_raw/helio_ecl_sph_00h/planets_dwarfs_daily_1849
Processing folder: ../../data/00_raw/helio_ecl_sph_00h/satellites_daily_1849
Processing completed. Merged data has 73780 rows and 624 columns.
Saved to: ../../data/ready/planets_satellites_lonlat.parquet
